In [1]:
from mcts import MCTS
from generator import Generator
from utils import Utils, Square
from backtrack import Backtrack
from state import State
from pieces import Piece

import time
import pandas as pd
import seaborn as sns

Generating puzzles

In [6]:
gen = Generator()
ps = [gen.getPuzzle(i) for i in range(8, 14) for _ in range(50)]
ps = {pid: p for pid, p in enumerate(ps)}
print(f"{len(ps)} puzzles to solve")

300 puzzles to solve


Creating a list of dictionaries which we'll turn into a dataframe later

In [10]:
data = []

Main loop of solving puzzles

In [11]:
for pid in ps:
    if pid < 276:
        continue
    print(f"solving puzzle {pid} out of {len(ps)}")

    # MCTS, Rank
    print(".")

    mcts = MCTS(ps[pid], h = "R")

    start = time.process_time()
    mcts.run()
    stop = time.process_time()
    
    t = stop - start
    ts = len(mcts.tree)
    vn = mcts.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"mcts",
        "visited_nodes":vn,
        "time":t,
        "tree_size":ts
    })

    # Backtrack, Rank, no treeset
    print(".")

    bt = Backtrack(ps[pid], h = "R", _tree = False)

    start = time.process_time()
    bt.run()
    stop = time.process_time()

    t = stop - start
    ts = len(bt.tree)
    vn = bt.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"bt",
        "visited_nodes":vn,
        "time":t,
        "tree_size":ts
    })

print("DONE")

solving puzzle 276 out of 300
.
.
solving puzzle 277 out of 300
.
.
solving puzzle 278 out of 300
.
.
solving puzzle 279 out of 300
.
.
solving puzzle 280 out of 300
.
.
solving puzzle 281 out of 300
.
.
solving puzzle 282 out of 300
.
.
solving puzzle 283 out of 300
.
.
solving puzzle 284 out of 300
.
.
solving puzzle 285 out of 300
.
.
solving puzzle 286 out of 300
.
.
solving puzzle 287 out of 300
.
.
solving puzzle 288 out of 300
.
.
solving puzzle 289 out of 300
.
.
solving puzzle 290 out of 300
.
.
solving puzzle 291 out of 300
.
.
solving puzzle 292 out of 300
.
.
solving puzzle 293 out of 300
.
.
solving puzzle 294 out of 300
.
.
solving puzzle 295 out of 300
.
.
solving puzzle 296 out of 300
.
.
solving puzzle 297 out of 300
.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\tools\Anaconda3\envs\env\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\xoort\AppData\Local\Temp\ipykernel_7628\161680230.py", line 12, in <module>
    mcts.run()
  File "c:\Users\xoort\Documents\Thesis\MCTS-SC\mcts.py", line 128, in run
    v = self.simulate(cur)
        ^^^^^^^^^^^^^^^^^^
  File "c:\Users\xoort\Documents\Thesis\MCTS-SC\mcts.py", line 165, in simulate
    node.getNexts()
  File "c:\Users\xoort\Documents\Thesis\MCTS-SC\mcts.py", line 43, in getNexts
    new_states = self.s.transition()
                 ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\xoort\Documents\Thesis\MCTS-SC\state.py", line 108, in transition
    actions = self.getActions()
              ^^^^^^^^^^^^^^^^^
  File "c:\Users\xoort\Documents\Thesis\MCTS-SC\state.py", line 93, in getActions
    if self.valCap(p1, p2):
       ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\xoort\Do

In [12]:
data = pd.DataFrame(data)
data.to_csv("./data_tree2.csv")